# 플레이스토어 어플 리뷰 크롤링
- 미소라는 어플의 리뷰를 최신순으로 크롤링
- 스크롤 내리면 리뷰들이 계속나오는데 리뷰들 계속 스크롤로 내릴 수 있도록 함

## 유튜브 참고
- https://www.youtube.com/watch?v=QGKLVzG0Jd8

## 주의사항
- 크롬에서 할때는 구글드라이버 작동이 조금 다를 수 있어서 다음과 같이 꼭 해야함 항상
  - 일단 밑에 두개 셀은 무슨작업을 하든 항상 고정이라고 생각
  - 그리고 driver = webdriver.Chrome('./chromedriver') 는 오류
    - driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options) 해줘야! 

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

In [2]:
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [4]:
from bs4 import BeautifulSoup
import time, os
from datetime import datetime
import pandas as pd

# review link link
link = 'https://play.google.com/store/apps/details?id=com.miso&hl=ko&showAllReviews=true'

scroll_count = 10 # 스크롤을 몇번 내릴것인가

driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
driver.get(link) # 드라이버에게 이 링크로 접속하게함

os.makedirs('result', exist_ok=True) # 결과를 담을 디렉토리를 만들어준다

for i in range(scroll_count):
  # 스크롤 동작수행
  driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
  time.sleep(3)

  # 더보기 버튼이 있으면 클릭한다
  try:
    load_more = driver.find_element_by_xpath('//*[contains(@class,"U26fgb O0WRkf oG5Srb C0oVfc n9lfJ")]').click()
  # 없는경우에도 잘 동작할수 있도록 예외처리(예외처리 안하면 오류난다)
  except:
    print(f'스크롤{i+1}회 완료, 더보기 버튼 없음')

# for문안에 스크롤이랑 같이 넣는 줄 알았는데 아니네.. 위치 주의!
reviews = driver.find_elements_by_xpath('//*[@jsname="fk8dgd"]//div[@class="d15Mdf bAhLNe"]') # 이걸 만족하는 애들을 reviews에 전부 담는다!

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


스크롤1회 완료, 더보기 버튼 없음
스크롤2회 완료, 더보기 버튼 없음
스크롤3회 완료, 더보기 버튼 없음
스크롤5회 완료, 더보기 버튼 없음
스크롤6회 완료, 더보기 버튼 없음
스크롤7회 완료, 더보기 버튼 없음
스크롤9회 완료, 더보기 버튼 없음
스크롤10회 완료, 더보기 버튼 없음


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead


In [6]:
print(f'reviews:{reviews}') # 이게 리스트형태임.

reviews:[<selenium.webdriver.remote.webelement.WebElement (session="352b1ba14e8c18545c7abae22ca4a418", element="15a58f5c-2c2d-4132-a036-999909d451e1")>, <selenium.webdriver.remote.webelement.WebElement (session="352b1ba14e8c18545c7abae22ca4a418", element="61947644-23a1-4735-b7e7-44e8dc3193ee")>, <selenium.webdriver.remote.webelement.WebElement (session="352b1ba14e8c18545c7abae22ca4a418", element="58de1ef7-6a5f-4c20-bd45-8d3f0da17d02")>, <selenium.webdriver.remote.webelement.WebElement (session="352b1ba14e8c18545c7abae22ca4a418", element="adb6b7a1-dde7-4f53-979e-2f5e04bef1d0")>, <selenium.webdriver.remote.webelement.WebElement (session="352b1ba14e8c18545c7abae22ca4a418", element="595b9aae-ecae-4401-b6ad-7f14953dcece")>, <selenium.webdriver.remote.webelement.WebElement (session="352b1ba14e8c18545c7abae22ca4a418", element="f65583ca-35fa-4db2-873f-1f6b7017e237")>, <selenium.webdriver.remote.webelement.WebElement (session="352b1ba14e8c18545c7abae22ca4a418", element="4e6ab989-cf16-433b-a346-

In [7]:
print('%d개의 리뷰들을 불러 올 수 있습니다' % len(reviews)) # 몇개의 리뷰를 긁어올 수 있는지 프린트

440개의 리뷰들을 불러 올 수 있습니다


In [8]:
# create empty dataframe to store data
df = pd.DataFrame(columns=['name', 'ratings', 'date', 'helpful', 'comment', 'developer_comment'])
df

,name,ratings,date,helpful,comment,developer_comment


In [9]:
# get review data
for review in reviews:
  # bs4를 이용해서 html요소 작업을 하려면 먼저 이렇게 해줘야함
  soup = BeautifulSoup(review.get_attribute('innerHTML'), 'html.parser')

  # reviewer
  name = soup.find(class_='X43Kjb').text

  # 이거 같은경우에는 img라는걸 보면 문장으로 되어있어서, 거기 텍스트 다 자르고 숫자만 갖고오는 방법임
  ratings = int(soup.find('div', role='img').get('aria-label').replace('별표 5개 만점에', '').replace('개를 받았습니다.', '').strip())

  # review date
  date = soup.find(class_='p2TkOb').text
  date = datetime.strptime(date, '%Y년 %m월 %d일')
  date = date.strftime('%Y-%m-%d')

  # helpful
  helpful = soup.find(class_='jUL89d y92BAb').text
  if not helpful: # 값이 없는 경우 0으로 처리
    helpful = 0
  
  # review text
  comment = soup.find('span', jsname='fbQN7e').text
  if not comment:
    comment = soup.find('span', jsname='bN97Pc').text
  
  # developer comment
  developer_comment = None
  dc_div = soup.find('div', class_='LVQB0b')
  if dc_div:
    developer_comment = dc_div.text.replace('\n', ' ')
  
  # append to dataframe
  df = df.append({
    'name': name,
    'ratings': ratings,
    'date': date,
    'helpful': helpful,
    'comment': comment,
    'developer_comment': developer_comment
  }, ignore_index=True)

# finally save the dataframe into csv file
filename = datetime.now().strftime('result/%Y-%m-%d_%H-%M-%S.csv')
df.to_csv(filename, encoding='utf-8-sig', index=False)
driver.stop_client()
driver.close()

print('크롤링 전체 완료')

크롤링 전체 완료


In [11]:
pd.read_csv('/content/result/2022-01-12_05-43-14.csv')
# 데이터 480건

,name,ratings,date,helpful,comment,developer_comment
0,김영진,5,2021-12-23,44,귀찮아도 방문 견적 받길 잘 했네요. 이사견적 사이트에 정보 넣으면 집이 커서 금액...,"Miso, Inc.2021년 12월 24일안녕하세요, 김영진 고객님. 미소 이사 서..."
1,김김용,5,2021-12-20,43,사람들이 제일 많이 쓰는 어플이라고 해서 저도 집청소 할 때 이용했습니다. 어플 쓰...,"Miso, Inc.2021년 12월 21일안녕하세요, 김김용 고객님. 미소 서비스를..."
2,쿠키찡,4,2022-01-02,9,"전체적으로 좋아서 항상 이용하고 있지만, 문의하기가 그지 같아서 별 하나 빼요. 이...","Miso, Inc.2022년 1월 3일안녕하세요, 쿠키찡 고객님. 미소 서비스 이용..."
3,졍열의위니,5,2021-12-20,34,어플 대박이네요. 냉장고 정리. 저 사실 계절 바뀔 때 한 번 하고 있었는데 미소에...,"Miso, Inc.2021년 12월 21일안녕하세요, 정열의위니 고객님. 미소 서비..."
4,곽성미,5,2021-12-30,33,모든 면에서 만족스럽습니다. 이사 예산이 적은 편이라 견적 받은 거 중 저렴한 걸로...,"Miso, Inc.2021년 12월 31일안녕하세요, 곽성미 고객님. 미소 서비스를..."
...,...,...,...,...,...,...
435,임성훈,5,2022-01-04,25,미소이용하고 나서 쾌적한 삶 누리고 있어요,"Miso, Inc.2022년 1월 5일안녕하세요, 임성훈 고객님. 미소 서비스를 이..."
436,강준혁,5,2021-11-26,38,출장 갈 때 마다 펫시터님 부르고 있어요. 전에는 강아지 호텔에 맡기고 하느라 맘이...,"Miso, Inc.2021년 11월 29일안녕하세요, 강준혁 고객님. 미소 서비스를..."
437,seonghun lee,5,2022-01-01,22,여자 혼자서 하는 이사라서.. 조오금 걱정되고 했었는데.. 이사 도와주러 오신 기사...,"Miso, Inc.2022년 1월 3일안녕하세요, seonghun lee 고객님. ..."
438,BA BE,1,2021-12-07,0,클리너분이 못오겠다고 해서 당일아침에 취소되고.. 다시 잡으니 클리너가 아예 매칭이...,"Miso, Inc.2021년 12월 8일안녕하세요, BA BE 고객님. 미소 서비스..."
